# Big Data Analytics

## add names here:




### The aim of this project is to use the IMDB Dataset: (Link) and use data such as actors, director, genre, isAdult and runtime to predict how well a movie will perform. We use the boxoffice and userrating as measures of performance. We will train our model with 80% of the data and test it on the remaining 20%. We will also give our predicitons for upcoming movies that do not have any ratings/boxoffice yet.

### Our hypothesis is that actors & director will positively influence the performance while the variable runtime will negatively influence it. 



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.linear_model import Ridge,LinearRegression,Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline



# Load Data and Preprocessing

## title basics 

In [2]:
df = pd.read_csv("data/title.basics.tsv",sep="\t")
# Select all non tv Series
df = df[(df['endYear']=='\\N')]
df.drop(["endYear"],axis=1,inplace=True)
df = df[(df['titleType']=="movie")]
# replace all \N with NAN and subsequently drop them
df.replace("\\N",np.nan,inplace=True)
df.dropna(inplace=True)
# Convert data types to int where feasible
df['startYear'] = df['startYear'].astype(int)
df['isAdult'] = df['isAdult'].astype(int)
df['runtimeMinutes'] = df['runtimeMinutes'].astype(int)

df = df[(df['startYear']>=2000) & (df['startYear']<=2020)]

df.reset_index(inplace=True,drop=True)
df.to_csv("data/title.basics_preprocessed.csv",index=False)
df

/home/flo/anaconda3/envs/datascience/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,67,Drama
1,tt0016906,movie,Frivolinas,Frivolinas,0,2014,80,"Comedy,Musical"
2,tt0019996,movie,Hongxia,Hongxia,0,2011,94,Action
3,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
4,tt0036177,movie,Muhomatsu no issho,Muhomatsu no issho,0,2008,100,"Action,Adventure"
...,...,...,...,...,...,...,...,...
189856,tt9916362,movie,Coven,Akelarre,0,2020,92,"Adventure,Drama,History"
189857,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,123,Drama
189858,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57,Documentary
189859,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,100,Documentary


## title crew

In [4]:
import os
print(os.getcwd())
print(os.listdir('./data'))
df = pd.read_csv('./data/title.crew.tsv', sep= '\t', header=0)
df.drop(['writers'], axis=1, inplace=True)
print(df['directors'].isnull().values.any())
df.to_csv("data/title.crew_preprocessed.csv",index=False)
df

/home/flo/Documents/FS2021/Big Data Analytics/gitRepo/BigDataAnalytics
['boxOffice_preprocessed.csv', 'title.crew_preprocessed.csv', 'README.md', 'title.crew.tsv', 'title.basics.tsv', 'tconsts.csv', 'name.basics.tsv', 'title.akas.tsv', 'title.principals.tsv', 'title.basics_preprocessed.csv', 'archive', 'boxOffice.csv', 'title.ratings.tsv', 'title.ratings.csv']
False


,tconst,directors
0,tt0000001,nm0005690
1,tt0000002,nm0721526
2,tt0000003,nm0721526
3,tt0000004,nm0721526
4,tt0000005,nm0005690
...,...,...
7869181,tt9916848,"nm5519454,nm5519375"
7869182,tt9916850,"nm5519375,nm5519454"
7869183,tt9916852,"nm5519454,nm5519375"
7869184,tt9916856,nm10538645


# title.akas

In [7]:
df = pd.read_csv("data/title.akas.tsv","\t",usecols=['titleId','language','isOriginalTitle'])
df

/home/flo/anaconda3/envs/datascience/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,titleId,language,isOriginalTitle
0,tt0000001,\N,0
1,tt0000001,\N,0
2,tt0000001,\N,0
3,tt0000001,\N,0
4,tt0000001,\N,0
...,...,...,...
26152127,tt9916852,ja,0
26152128,tt9916852,pt,0
26152129,tt9916852,it,0
26152130,tt9916852,hi,0


In [8]:
df = df[df["isOriginalTitle"]==1]
np.unique(df["language"])

array(['\\N', 'ca', 'da', 'en', 'ja'], dtype=object)

## name basics 

In [5]:
df = pd.read_csv("data/name.basics.tsv",sep="\t")
# Drop unnecessary columns
df.drop(["birthYear", "deathYear","primaryProfession","knownForTitles"],axis=1,inplace=True)
# replace all \N with NAN and subsequently drop them
df.replace("\\N",np.nan,inplace=True)
df.dropna(inplace=True)

df.reset_index(inplace=True,drop=True)
df.to_csv("data/name.basics_preprocessed.csv",index=False)
df

,nconst,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
...,...,...
10900082,nm9993714,Romeo del Rosario
10900083,nm9993716,Essias Loberg
10900084,nm9993717,Harikrishnan Rajan
10900085,nm9993718,Aayush Nair


## title principals

In [6]:
df = pd.read_csv("data/title.principals.tsv",sep="\t")
# Drop unnecessary columns
df.drop(["ordering", "category","job","characters"],axis=1,inplace=True)
# replace all \N with NAN and subsequently drop them
df.replace("\\N",np.nan,inplace=True)
df.dropna(inplace=True)

df.reset_index(inplace=True,drop=True)
df.to_csv("data/title.principals_preprocessed.csv",index=False)
df

,tconst,nconst
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0374658
3,tt0000002,nm0721526
4,tt0000002,nm1335271
...,...,...
44505357,tt9916880,nm0996406
44505358,tt9916880,nm1482639
44505359,tt9916880,nm2586970
44505360,tt9916880,nm1594058


## title ratings

In [25]:
df = pd.read_csv("data/title.ratings.tsv", sep ="\t")
# Drop unnecessary columns
df.drop(["numVotes"], axis = 1, inplace = True)
df.dropna(inplace = True)

df.reset_index(inplace=True, drop=True)
df.to_csv("data/title.ratings_preprocessed.csv", index=False)
df

,tconst,averageRating
0,tt0000001,5.7
1,tt0000002,6.1
2,tt0000003,6.5
3,tt0000004,6.1
4,tt0000005,6.1
...,...,...
1148469,tt9916682,5.6
1148470,tt9916690,6.6
1148471,tt9916720,6.5
1148472,tt9916766,6.9


## Box Office Data

In [9]:
df = pd.read_csv("data/boxOffice.csv")
df.dropna(how="all",subset=["domestic","international","worldwide"],inplace=True)
df.reset_index(inplace=True,drop=True)
df.to_csv("data/boxOffice_preprocessed.csv",index=False)
df

# Joining Data

In [57]:
def join_another(df1,df2, on_col):
    df1['tconst'] = df1['tconst'].astype(str)
    df2['tconst'] = df2['tconst'].astype(str)
    return pd.merge(df1,df2,on=on_col,how="inner")


df = pd.read_csv("data/boxOffice_preprocessed.csv")

fileNames = [
    'title.basics_preprocessed.csv',
    'title.crew_preprocessed.csv',
    "title.principals_preprocessed.csv",
    "title.ratings_preprocessed.csv"
    ]
for fileName in fileNames:
    df_to_join = pd.read_csv("data/"+fileName)
    df = join_another(df,df_to_join,'tconst')

df_to_join = pd.read_csv("data/name.basics_preprocessed.csv")
df = pd.merge(df,df_to_join,how='inner',on="nconst")
df_to_join.rename(columns={"nconst":'directors'})
df = pd.merge(df,df_to_join,how='inner',on=["directors"])
df = df.rename(columns={"primaryName_x":"actor_name","primaryName_y":"director_name"})
df.to_csv("data/allData.csv")
df


,tconst,domestic,international,worldwide,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,nconst,averageRating
0,tt0035423,47121859.0,28897189.0,76019048.0,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,nm0107463,6.4
1,tt0035423,47121859.0,28897189.0,76019048.0,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,nm0000212,6.4
2,tt0035423,47121859.0,28897189.0,76019048.0,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,nm0413168,6.4
3,tt0035423,47121859.0,28897189.0,76019048.0,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,nm0000630,6.4
4,tt0035423,47121859.0,28897189.0,76019048.0,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",nm0003506,nm0005227,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335759,tt9905412,NaN,4791.0,4791.0,movie,Ottam,Ottam,0,2019,120,Drama,nm10533890,nm10533890,7.3
335760,tt9905412,NaN,4791.0,4791.0,movie,Ottam,Ottam,0,2019,120,Drama,nm10533890,nm10533895,7.3
335761,tt9905412,NaN,4791.0,4791.0,movie,Ottam,Ottam,0,2019,120,Drama,nm10533890,nm6546398,7.3
335762,tt9905412,NaN,4791.0,4791.0,movie,Ottam,Ottam,0,2019,120,Drama,nm10533890,nm6412514,7.3


# Preprocessing whole data set

## One hot encode genres

In [9]:
df = pd.read_csv("data/allData.csv")
genres = np.unique(df['genres'])
genreset = set()
for genre in genres:
    for g in genre.split(","):
        genreset.add(g)

print(genreset)
print(len(genreset))
for i in genreset:
    df[i] = 0

{'Crime', 'Thriller', 'Horror', 'Western', 'Musical', 'Adult', 'Animation', 'Action', 'Mystery', 'Documentary', 'History', 'Fantasy', 'Family', 'Biography', 'War', 'Adventure', 'Comedy', 'Sport', 'Drama', 'Sci-Fi', 'Music', 'Romance', 'News'}
23


In [96]:
df[df['startYear']>=2017].shape

(72183, 39)

In [101]:
for i in df.columns:
    print(f"{i}: {np.sum(df[i].isnull())}")
df.iloc[0]

tconst: 0
domestic: 204539
international: 27259
worldwide: 0
titleType: 0
primaryTitle: 0
originalTitle: 0
isAdult: 0
startYear: 0
runtimeMinutes: 0
genres: 0
directors: 0
nconst: 0
averageRating: 0
actor_name: 0
director_name: 0
Action: 0
Fantasy: 0
Comedy: 0
History: 0
Animation: 0
Sport: 0
Biography: 0
Documentary: 0
Crime: 0
Western: 0
Adult: 0
Romance: 0
Musical: 0
War: 0
Mystery: 0
Drama: 0
Family: 0
Sci-Fi: 0
Music: 0
Horror: 0
Thriller: 0
News: 0
Adventure: 0


tconst                         tt0035423
domestic                      47121859.0
international                 28897189.0
worldwide                     76019048.0
titleType                          movie
primaryTitle              Kate & Leopold
originalTitle             Kate & Leopold
isAdult                                0
startYear                           2001
runtimeMinutes                       118
genres            Comedy,Fantasy,Romance
directors                      nm0003506
nconst                         nm0107463
averageRating                        6.4
actor_name                 David Brenner
director_name              James Mangold
Action                                 0
Fantasy                                1
Comedy                                 1
History                                0
Animation                              0
Sport                                  0
Biography                              0
Documentary                            0
Crime           

In [16]:
x = df[df['primaryTitle']=="Kate & Leopold"]
np.unique(x['actor_name'])

array(['Breckin Meyer', 'Cathy Konrad', 'David Brenner', 'Hugh Jackman',
       'James Mangold', 'Liev Schreiber', 'Meg Ryan', 'Rolfe Kent',
       'Steven Rogers', 'Stuart Dryburgh'], dtype=object)

# Data Preprocessing

## Steps performed

### dropping columns, imputing missing values, categorical datapoints


# Kfold to decrease potential overfitting

In [ ]:
from sklearn.model_selection import KFold

def kfold_validation(X,y,model):
    start = time.time()
    kf = KFold(n_splits=5,shuffle=True, random_state=69420) 
    mses = []
    models = []
    count = 0
    for trainIndices,testIndices in kf.split(X):
        print(f"iteration: {count}")
        Xtrain,Xval = X[trainIndices,:],X[testIndices,:]
        ytrain,yval = y[trainIndices],y[testIndices]
        model.fit(Xtrain,ytrain)
        yhat = model.predict(Xval)
        mse = np.sum(np.square(yval-yhat))/yval.size # mean squared error
        mses.append(mse)
        models.append(model)
        count+=1
    print(f"time used (seconds): {time.time()-start}")
    return mses, models


# Standardise to make data more comparable

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Xtrain,Xtest, ytrain,ytest = train_test_split(X,y,shuffle=True,train_size=0.8)

def standardise_data(X):
    mean = np.mean(X)
    std = np.std(X)
    X_std = (X-mean)/std
    return X_std, mean, std

#kf = KFold(n_splits=1,shuffle=True, random_state=69420) 

Xtrain_std, X_train_mean, X_train_std_div = standardise_data(Xtrain)
Xtest_std = (Xtest-X_train_mean)/X_train_std_div


# Linear Regression

In [ ]:
model = LinearRegression()
mses,models = kfold_validation(Xtrain,ytrain,model)
yhat=models[np.argmin(mses)].predict(Xtest)
np.mean(np.square(yhat-ytest))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor 

model = RandomForestRegressor()
mses,models = kfold_validation(Xtrain,ytrain,model)
yhat=models[np.argmin(mses)].predict(Xtest)
np.mean(np.square(yhat-ytest))


# Data Visualisation